In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from tqdm import tqdm
import torchvision.transforms.functional as TF


In [2]:
# Configuration
fmri_root_folder = r'C:\Users\SAIFUL_BADHON\Downloads\fMRI\output'  # Top-level folder containing subfolders
csv_path = r'C:\Users\SAIFUL_BADHON\Downloads\fMRI_3_24_2025.csv'
img_size = (64, 64, 64)
batch_size = 8
num_epochs = 50
lr = 0.001

In [3]:

# Step 1: Load metadata
df = pd.read_csv(csv_path)
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['Group'])  # Converts Group to 0-5 labels


In [4]:
def check_valid_nii_exists(image_id, root_folder):
    folder = os.path.join(root_folder, str(image_id))
    if not os.path.isdir(folder):
        return False
    for file in os.listdir(folder):
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            return True
    return False

df['has_file'] = df['Image Data ID'].apply(lambda x: check_valid_nii_exists(x, fmri_root_folder))
df = df[df['has_file']].reset_index(drop=True)

def map_to_binary(group):
    if group in ['CN', 'SMC']:
        return 0
    else:
        return 1

df['label'] = df['Group'].apply(map_to_binary)
# # ✅ Step 2: Encode labels *after* filtering so they match the actual data
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['Group'])

# Debug check
# print("Final label classes used:", list(label_encoder.classes_))
print("Encoded label values:", df['label'].unique())

Encoded label values: [1 0]


In [5]:
df

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded,has_file,label
0,I1178798,007_S_6341,MCI,M,68,y1,fMRI,Axial MB rsfMRI (Eyes Open),Original,6/11/2019,DCM,NaN,True,1
1,I990573,007_S_6341,MCI,M,67,sc,fMRI,Axial MB rsfMRI (Eyes Open),Original,4/30/2018,DCM,NaN,True,1
2,I1327196,007_S_6310,CN,F,70,y2,fMRI,Axial MB rsfMRI (Eyes Open),Original,8/05/2020,DCM,NaN,True,0
3,I974760,007_S_6255,CN,F,75,sc,fMRI,Axial MB rsfMRI (Eyes Open),Original,3/05/2018,DCM,NaN,True,0
4,I1325573,007_S_6255,CN,F,78,y2,fMRI,Axial MB rsfMRI (Eyes Open),Original,7/27/2020,DCM,NaN,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,I243902,002_S_0685,CN,F,95,v06,fMRI,Resting State fMRI,Original,7/08/2011,DCM,NaN,True,0
217,I1221056,002_S_0413,CN,F,90,y2,fMRI,Axial MB rsfMRI (Eyes Open),Original,8/27/2019,DCM,NaN,True,0
218,I304790,002_S_0413,CN,F,82,v11,fMRI,Resting State fMRI,Original,5/15/2012,DCM,NaN,True,0
219,I240811,002_S_0413,CN,F,82,v06,fMRI,Resting State fMRI,Original,6/16/2011,DCM,NaN,True,0


In [6]:
df["Group"].value_counts()

Group
CN      104
MCI      41
LMCI     32
EMCI     25
AD       12
SMC       7
Name: count, dtype: int64

In [7]:
df["label"].value_counts()

label
0    111
1    110
Name: count, dtype: int64

In [8]:
# Step 2: Preprocess fMRI images (4D -> 3D)
def load_and_preprocess_nifti_from_folder(folder_path):
    nii_file = None
    for file in os.listdir(folder_path):
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            nii_file = os.path.join(folder_path, file)
            break
    if nii_file is None:
        raise FileNotFoundError(f"No NIfTI file found in {folder_path}")
    img = nib.load(nii_file)
    data = img.get_fdata()
    mean_3d = np.mean(data, axis=3)  # 4D -> 3D
    return mean_3d

def resize_volume(img, size=img_size):
    import scipy.ndimage
    zoom_factors = [s / float(img.shape[i]) for i, s in enumerate(size)]
    return scipy.ndimage.zoom(img, zoom=zoom_factors, order=1)

In [17]:
# Step 3: PyTorch Dataset
class FMRIDataset(Dataset):
    def __init__(self, dataframe, data_root, transform=None):
        self.df = dataframe
        self.data_root = data_root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = str(row['Image Data ID'])
        label = int(row['label'])
        folder_path = os.path.join(self.data_root, img_id)
        volume = load_and_preprocess_nifti_from_folder(folder_path)
        volume = resize_volume(volume)
        volume = (volume - volume.mean()) / (volume.std() + 1e-5)
        volume = np.expand_dims(volume, axis=0)  # Shape: (1, D, H, W)
        return torch.tensor(volume, dtype=torch.float32), torch.tensor(label)

In [19]:
# Step 4: Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_dataset = FMRIDataset(train_df, fmri_root_folder)
test_dataset = FMRIDataset(test_df, fmri_root_folder)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3DEncoder(nn.Module):
    def __init__(self, output_size=128):
        super(CNN3DEncoder, self).__init__()
        self.conv1 = nn.Conv3d(1, 8, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool3d(2)
        self.conv2 = nn.Conv3d(8, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool3d(2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 16 * 16 * 16, output_size)  # Adjust if your input volume shape changes

    def forward(self, x):  # x: [B, 1, D, H, W]
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x  # [B, output_size]
class CNN_LSTM(nn.Module):
    def __init__(self, cnn_embed_dim=128, lstm_hidden_dim=64, num_layers=1, num_classes=2):
        super(CNN_LSTM, self).__init__()
        self.cnn_encoder = CNN3DEncoder(output_size=cnn_embed_dim)
        self.lstm = nn.LSTM(input_size=cnn_embed_dim,
                            hidden_size=lstm_hidden_dim,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, num_classes)

    def forward(self, x):  # x: [B, T, 1, D, H, W]
        batch_size, T, C, D, H, W = x.size()
        x = x.view(batch_size * T, C, D, H, W)
        cnn_features = self.cnn_encoder(x)  # [B*T, cnn_embed_dim]
        cnn_features = cnn_features.view(batch_size, T, -1)  # [B, T, cnn_embed_dim]
        lstm_out, _ = self.lstm(cnn_features)  # [B, T, lstm_hidden_dim]
        out = self.fc(lstm_out[:, -1, :])  # Use the last time step
        return out


In [23]:

# Step 6: Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_LSTM(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader):.4f}")



Epoch 1/50:   0%|                                                                               | 0/22 [00:08<?, ?it/s]


ValueError: not enough values to unpack (expected 6, got 5)

In [55]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Report precision, recall, f1-score for each class

report = classification_report(all_labels, all_preds)
print("Classification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.49      1.00      0.66        22

    accuracy                           0.49        45
   macro avg       0.24      0.50      0.33        45
weighted avg       0.24      0.49      0.32        45



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
